In [1]:
import json, requests

## Clean Surah Object

In [3]:
with open('surahAdj.json', 'rb') as file:
    f = json.load(file)
f = dict(f)

In [6]:
newClean = []
for ch in f['chapters']:
    newClean.append({
        'makkah': bool(ch['revelation_place'] == 'makkah'),
        'name_simple': ch['name_simple'],
        'name_arabic': ch['name_arabic'],
        'verses_count': ch['verses_count'],
    })
newClean = {'chapters': newClean}
newClean['chapters'][:2]

[{'makkah': False,
  'name_simple': 'Al-Fatihah',
  'name_arabic': 'الفاتحة',
  'verses_count': 7},
 {'makkah': False,
  'name_simple': 'Al-Baqarah',
  'name_arabic': 'البقرة',
  'verses_count': 286}]

In [7]:
with open('surahAdj.json', 'w') as file:
    json.dump(newClean, file)

## Quran Pages

In [19]:
all_pages = []
for page_num in range(1, 605):
    obj = {}
    res = requests.get(f'https://api.quran.com/api/v4/verses/by_page/{page_num}')
    js = res.json()
    for i, v in enumerate(js['verses']):
        # print(i, v)
        js['verses'][i] = {
            'verse_key': v['verse_key'],
            'juz_number': v['juz_number'],
        }
    all_pages.append(js['verses'])


In [21]:
all_pages[:1]

[[{'verse_key': '1:1', 'juz_number': 1},
  {'verse_key': '1:2', 'juz_number': 1},
  {'verse_key': '1:3', 'juz_number': 1},
  {'verse_key': '1:4', 'juz_number': 1},
  {'verse_key': '1:5', 'juz_number': 1},
  {'verse_key': '1:6', 'juz_number': 1},
  {'verse_key': '1:7', 'juz_number': 1}]]

In [22]:
with open('versesPerPage.json', 'w') as file:
    json.dump({"pages": all_pages}, file)